In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 739.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 59.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=5)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            # Handle the case where logits are missing, e.g., raise an exception or return a default loss
            raise ValueError("Logits are missing from the model output.")  # Or return a default loss

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2024-12-10 17:54:06,862] A new study created in memory with name: no-name-d8799fc7-7cbe-4d4a-8e98-4f03152a6a5d
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.556189,0.775597,0.222965,0.045131,0.068701
1,No log,0.518736,0.776658,0.000000,0.000000,0.000000
2,No log,0.507797,0.776658,0.000000,0.000000,0.000000
3,No log,0.501850,0.776658,0.000000,0.000000,0.000000
4,No log,0.498382,0.777188,0.258907,0.002375,0.004707
5,0.527400,0.497203,0.777188,0.258907,0.002375,0.004707


***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 377
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-10 17:56:12,059] Trial 0 finished with value: 0.004707406607644137 and parameters: {'learning_rate': 1.1333372015346764e-06, 'batch_size': 8, 'num_train_epochs': 6, 'weight_decay': 0.09677223325184231}. Best is trial 0 with value: 0.004707406607644137.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This fe

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.481186,0.783024,0.484824,0.030879,0.057767
1,No log,0.459096,0.815915,0.450729,0.204276,0.280637
2,No log,0.443788,0.824403,0.593403,0.261283,0.328777
3,No log,0.434025,0.826525,0.578413,0.287411,0.348314
4,No log,0.427330,0.830769,0.573312,0.296912,0.371668
5,No log,0.422987,0.831300,0.569109,0.304038,0.375068
6,No log,0.421626,0.834483,0.577542,0.315914,0.390816


...

[I 2024-12-10 18:12:41,087] Trial 8 finished with value: 0.7419519568013077 and parameters: {'learning_rate': 2.3990739528107452e-05, 'batch_size': 8, 'num_train_epochs': 3, 'weight_decay': 0.009198763795425122}. Best is trial 7 with value: 0.7439991294330432.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: T

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.575392,0.884881,0.760625,0.717340,0.733718
1,No log,0.574867,0.889125,0.764793,0.731591,0.743855
2,0.003300,0.580577,0.888064,0.764800,0.729216,0.742621
3,0.003300,0.576782,0.890716,0.767616,0.736342,0.747640
4,0.003300,0.584068,0.890186,0.769660,0.729216,0.745058
5,0.002000,0.586766,0.890186,0.768280,0.731591,0.745563
6,0.002000,0.590947,0.890186,0.768305,0.731591,0.745733
7,0.001900,0.584793,0.892838,0.772383,0.738717,0.751796
8,0.001900,0.583422,0.892838,0.771004,0.741093,0.752357
9,0.001900,0.583574,0.892838,0.773284,0.738717,0.751698


***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-

[I 2024-12-10 18:16:43,630] Trial 9 finished with value: 0.7516976795929688 and parameters: {'learning_rate': 1.5311063127923238e-06, 'batch_size': 4, 'num_train_epochs': 10, 'weight_decay': 0.0009136184558874768}. Best is trial 9 with value: 0.7516976795929688.


Best trial:
FrozenTrial(number=9, state=TrialState.COMPLETE, values=[0.7516976795929688], datetime_start=datetime.datetime(2024, 12, 10, 18, 12, 41, 89206), datetime_complete=datetime.datetime(2024, 12, 10, 18, 16, 43, 630072), params={'learning_rate': 1.5311063127923238e-06, 'batch_size': 4, 'num_train_epochs': 10, 'weight_decay': 0.0009136184558874768}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=9, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"<<==============Training Time==================>>")
print(f"Training time: {training_time:.2f} seconds")
print(f"<<=============================================>>")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 1880


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.604027,0.891247,0.773249,0.731591,0.746998
1,No log,0.605163,0.889125,0.765046,0.729216,0.743468
2,0.001400,0.608581,0.890186,0.767075,0.733967,0.746165
3,0.001400,0.614358,0.891777,0.771256,0.736342,0.749414
4,0.001400,0.629005,0.888064,0.765033,0.724466,0.739855
5,0.001200,0.630845,0.889655,0.769150,0.729216,0.744126
6,0.001200,0.625477,0.891247,0.770698,0.733967,0.747912
7,0.001100,0.628856,0.890186,0.769116,0.729216,0.744248
8,0.001100,0.627194,0.890186,0.768084,0.731591,0.745537
9,0.001100,0.627560,0.891247,0.771425,0.731591,0.747239


***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1500
Configuration saved in ./best_results/checkpoint-1500/config.json
Mode

<<==============Training Time==================>>
Training time: 240.03 seconds
<<=============================================>>


In [13]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 377
  Batch size = 8


{'eval_loss': 0.6275604963302612, 'eval_accuracy': 0.8912466843501327, 'eval_precision': 0.7714247565564057, 'eval_recall': 0.7315914489311164, 'eval_f1': 0.747239261766884, 'eval_runtime': 1.5696, 'eval_samples_per_second': 240.189, 'eval_steps_per_second': 30.581, 'epoch': 10.0}


### Model Saving

In [14]:
!mkdir 'best_model_scibert_scivocab_uncased_python'
!mkdir 'best_model_scibert_scivocab_uncased_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_scibert_scivocab_uncased_python')
tokenizer.save_pretrained('./best_model_scibert_scivocab_uncased_python_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_scibert_scivocab_uncased_python')
tokenizer = AutoTokenizer.from_pretrained('./best_model_scibert_scivocab_uncased_python_tokenizer')

Saving model checkpoint to ./best_model_scibert_scivocab_uncased_python
Configuration saved in ./best_model_scibert_scivocab_uncased_python/config.json
Model weights saved in ./best_model_scibert_scivocab_uncased_python/pytorch_model.bin
tokenizer config file saved in ./best_model_scibert_scivocab_uncased_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_scibert_scivocab_uncased_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_scibert_scivocab_uncased_python/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_scibert_scivocab_uncased_python",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermedia

### Model Testing

In [15]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [16]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

[array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 1, 0, 0, 0]) array([0, 0, 1, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([1, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 1, 0, 0])
 array([0, 0, 0, 0, 

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

# Display metrics and inference time
print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 406
  Batch size = 8


Evaluation Metrics: {'accuracy': 0.8719211822660099, 'precision': 0.7055712273286685, 'recall': 0.6674311926605505, 'f1': 0.6838155100816556}
Inference Time: 1.79 seconds


In [18]:
!zip -r best_model_scibert_scivocab_uncased_python.zip './best_model_scibert_scivocab_uncased_python'
!zip -r best_model_scibert_scivocab_uncased_python_tokenizer.zip './best_model_scibert_scivocab_uncased_python_tokenizer'

  adding: best_model_scibert_scivocab_uncased_python/ (stored 0%)
  adding: best_model_scibert_scivocab_uncased_python/pytorch_model.bin (deflated 7%)
  adding: best_model_scibert_scivocab_uncased_python/config.json (deflated 52%)
  adding: best_model_scibert_scivocab_uncased_python/training_args.bin (deflated 51%)
  adding: best_model_scibert_scivocab_uncased_python_tokenizer/ (stored 0%)
  adding: best_model_scibert_scivocab_uncased_python_tokenizer/tokenizer.json (deflated 71%)
  adding: best_model_scibert_scivocab_uncased_python_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_scibert_scivocab_uncased_python_tokenizer/tokenizer_config.json (deflated 43%)
  adding: best_model_scibert_scivocab_uncased_python_tokenizer/vocab.txt (deflated 52%)


In [19]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_scibert_scivocab_uncased_python.zip "/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python/"
!cp best_model_scibert_scivocab_uncased_python_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python_Tokenizer/"

Mounted at /content/drive


### Load and Test Model

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python/best_model_scibert_scivocab_uncased_python.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python_Tokenizer/best_model_scibert_scivocab_uncased_python_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python/best_model_scibert_scivocab_uncased_python.zip
replace ./best_model_scibert_scivocab_uncased_python/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_scibert_scivocab_uncased_python/pytorch_model.bin  
  inflating: ./best_model_scibert_scivocab_uncased_python/config.json  
  inflating: ./best_model_scibert_scivocab_uncased_python/training_args.bin  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/scibert_scivocab_uncased_Python_Tokenizer/best_model_scibert_scivocab_uncased_python_tokenizer.zip
replace ./best_model_scibert_scivocab_uncased_python_tokenizer/tokenizer.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_scibert_scivocab_uncased_python_tokenizer/tokenizer.json  
  inflating: ./best_model_scibert_scivocab_uncased_python_tokenizer/special_tokens_map.json  
  inflating: ./best_model_scibert_scivocab_uncased_python_tokenizer/tokenizer_co

In [22]:
best_model_scibert_scivocab_uncased_python = 'best_model_scibert_scivocab_uncased_python'
best_model_scibert_scivocab_uncased_python_tokenizer = 'best_model_scibert_scivocab_uncased_python_tokenizer'

In [23]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_scibert_scivocab_uncased_python)
tokenizer = AutoTokenizer.from_pretrained(best_model_scibert_scivocab_uncased_python_tokenizer)

loading configuration file best_model_scibert_scivocab_uncased_python/config.json
Model config BertConfig {
  "_name_or_path": "best_model_scibert_scivocab_uncased_python",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 3109

In [24]:
text = "for detecting automatically generated fields."
start_time = time.time()
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
end_time = time.time()
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.2f} seconds")
output

SequenceClassifierOutput(loss=None, logits=tensor([[-7.0553, -6.9720, -6.4482, -7.4082,  7.0523]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [25]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 5
